**Assignment Week 14**

Due: 5/6/2024

Shimon Greengart

First, I get the data. I'm not sure why it takes so much longer to get this dataset than all previous datasets. I make my train-test split.

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import pandas as pd
import keras as kr
import gensim as gs
import gensim.downloader as gs_api
from sklearn import feature_extraction, model_selection

imdb, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_df, test_df = tfds.as_dataframe(imdb['train'], info), tfds.as_dataframe(imdb['test'], info)

train_data = train_df.to_numpy()
test_data = test_df.to_numpy()

X_train, y_train = train_data[:, 1], train_data[:, 0].astype('float32')
X_test, y_test = test_data[:, 1], test_data[:, 0].astype('float32')

# train-validation-test split
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, shuffle=True)

train_df.head()
print(X_train[:5])
print(y_train[:5])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU2VCRR/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU2VCRR/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU2VCRR/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
[b'I don\'t know...Maybe it\'s just because it\'s an impressive tribute to some Muslim religious action(hajj)but I just felt the movie is so underrated. I just can\'t believe that the movie has just been voted by only 223 people so far given that the movie was produced in 2004 and it has won many awards since then.About the movie...it\'s one of those well-acted sweet movies.Reda,a French teenager due to sit for Baccalaur\xc3\xa9at, is asked by his devout elderly father to take him to Mecca.Strange as it may seem(if one doesn\'t know much about Islam)the father wants his son to drive them from their home in France to Saudia Arabia on a once-in-a-lifetime religious pilgrimage.The generation gap between the father and the son is based on simple enough terms(\'you may know how to read and write, but you know nothing about life,\' the unnamed father

This preprocessing I do for both my bag of words model and my aggregation model.

In [2]:
# Note: You might want to do something about the weird line breaks
# And you might want other preprocessing
# But this should be fine for now
# Before I go onward, I'm going to lowercase everything
print(X_train.shape)
vectorizer = feature_extraction.text.CountVectorizer(input='content')
X_train_bow = vectorizer.fit_transform(X_train).astype('float32')
X_val_bow = vectorizer.transform(X_val).astype('float32')
X_test_bow = vectorizer.transform(X_test).astype('float32')
print(X_train_bow.shape)

X_train_bow.sort_indices()
X_val_bow.sort_indices()
X_test_bow.sort_indices()

print(type(X_train_bow))

(20000,)
(20000, 68549)
<class 'scipy.sparse._csr.csr_matrix'>


This is my model built on a bag-of-words model. I use sklearn's CountVectorizer to create a bag-of-words that provides a number for each word. Conviniently, it produces this as a sparse matrix, allowing me to do this without blowing up my RAM (a recurring problem in this assignment). Then I use a very simple neural network with a single hidden layer to achieve over 89% accuracy.

Its F1 score is surprisingly bad, worse than both precision and accuracy. I'm not sure why F1 is consistently bad and similar across the three models. There may be something wrong in the way I am using Keras's metric. When I calculated it manually, I got 0.897, significantly higher. There may be a bug in their code.

In [5]:
# Here we have a model
# FEATURE_SHAPE = X_train_bow[0].shape
HIDDEN_SIZE = 300
LEARNING_RATE = 0.0001
NUM_EPOCHS = 50

bow_model = kr.models.Sequential()
# bow_model.add(kr.Input(shape=FEATURE_SHAPE))  # It doesn't like my shape for some reason
bow_model.add(kr.layers.Dense(HIDDEN_SIZE, activation='relu'))
bow_model.add(kr.layers.Dense(1, activation='sigmoid'))

bow_model.compile(optimizer=kr.optimizers.Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy',
                 metrics=[kr.metrics.BinaryAccuracy(), kr.metrics.Precision(), kr.metrics.Recall(), kr.metrics.F1Score()])
bow_model.fit(X_train_bow, y_train, epochs=NUM_EPOCHS, validation_data=(X_val_bow, y_val),
              callbacks=[kr.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=1, start_from_epoch=1)])


Epoch 1/50
625/625 [==============================] - 84s 132ms/step - loss: 0.3784 - binary_accuracy: 0.8559 - precision_1: 0.8601 - recall_1: 0.8517 - f1_score: 0.6686 - val_loss: 0.2905 - val_binary_accuracy: 0.8940 - val_precision_1: 0.8583 - val_recall_1: 0.9393 - val_f1_score: 0.6588
Epoch 2/50
625/625 [==============================] - 46s 74ms/step - loss: 0.1828 - binary_accuracy: 0.9427 - precision_1: 0.9407 - recall_1: 0.9455 - f1_score: 0.6688 - val_loss: 0.2794 - val_binary_accuracy: 0.8962 - val_precision_1: 0.8634 - val_recall_1: 0.9369 - val_f1_score: 0.6591
Epoch 3/50
625/625 [==============================] - 28s 45ms/step - loss: 0.1105 - binary_accuracy: 0.9706 - precision_1: 0.9697 - recall_1: 0.9718 - f1_score: 0.6695 - val_loss: 0.2757 - val_binary_accuracy: 0.8994 - val_precision_1: 0.9027 - val_recall_1: 0.8913 - val_f1_score: 0.6608
Epoch 4/50
625/625 [==============================] - 19s 30ms/step - loss: 0.0700 - binary_accuracy: 0.9850 - precision_1: 0.984

This is my second version that uses an embedding with a neural network. You said we shouldn't use standard doc2vec, but instead make our own aggregations.


First, I get the model. I decided to use a pretrained word2vec model rather than make my own, which was perhaps a mistake. I originally tried using a Wikipedia model, but found that there were too many words not used in it. The problem persists in the twitter model I ultimately settled with, but not as bad as it was.

In [7]:
model = gs_api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


Next, I do some more preprocessing and then run the model. This is by far my worst model, only reaching 82% accuracy. There are multiple reasons I identified.

The first of these is all the missing words not in the Twitter model. The lack of these words probably means a lack of meaning that overrides the additional meaning added by word2vec.

I think another reason for its poor performance is that it's not using all the data. It's based on the bag-of-words, but I averaged all words found in a review equally, not by count. The reason is because common words like "that" would overwhelm the average. I could have used something like TF-IDF to correct for that problem, but I decided not to to leave this model as a better comparison.

But there is one measurement in which this model does *much* better than the previous one. Namely, it only takes only a minute or two to train, unlike my previous model which, without a graphics card, took over 18 minutes. This is because rather than having to look at massive word-count arrays, it only has to look at the average vector. This might carry much less information, but it is also less information to have to go through.

In [26]:
# Now, I want to make an aggregation, starting with the mean. You know what, I'm tired of this. Let's stop for now.
# Okay, so, things will take forever, because I have apparently run out of GPU time without paying.
# But anyway, I should figure out how to ammalgamate the vectors.
# I'm not going to weight it based on number of appearances, because that would overly heighten the weights of what probably should be stopwords.
# Maybe I should have used TF-IDF.
# Whatever, let's start by generating a reverse lookup of the the dictionary based on the bag-of-words.
word_by_index = {index:word for word, index in vectorizer.vocabulary_.items()}
# now we generate the aggregation for each thing
# We go through each index present and find its word
# Then we throw that word through our gensim word2vec to get its embedding
# Then we add that embedding to our current sum for the document
# let's make it a function, so I can easily apply it to everything
def make_vector_aggregate(bags_of_words):
    bad_word_set = set()
    def aggregate_single_bow(sparse_bow):
        aggr_array = np.zeros(model.vector_size)
        total_words = len(sparse_bow.indices)
        for index in sparse_bow.indices:
            word = word_by_index[index]
            # Note: Not every word here is in my model...
            # For example, it doesn't have 'artsy'
            # Hopefully, that's not a problem
            if word in model:
                vector = model[word]
                aggr_array += vector
            else:
                bad_word_set.add(word)
                total_words -= 1
                # print(bad_word_set)
                # print(word)
        aggr_array /= len(sparse_bow.indices)
        return aggr_array

    # print(bad_word_set)
    ret_val = np.array([aggregate_single_bow(sparse_bow) for sparse_bow in bags_of_words])
    print(f'Set length is {len(bad_word_set)}')
    return ret_val


X_train_aggr = make_vector_aggregate(X_train_bow)
X_val_aggr = make_vector_aggregate(X_val_bow)
X_test_aggr = make_vector_aggregate(X_test_bow)

print(X_train_aggr[0])

# And now the model
EMBED_FEATURE_SHAPE = X_train_aggr[0].shape
EMBED_HIDDEN_SIZE = 50
EMBED_LEARNING_RATE = 0.005
EMBED_NUM_EPOCHS = 50

aggr_model = kr.models.Sequential()
aggr_model.add(kr.Input(shape=EMBED_FEATURE_SHAPE))
aggr_model.add(kr.layers.Dense(EMBED_HIDDEN_SIZE, activation='relu'))
aggr_model.add(kr.layers.Dense(1, activation='sigmoid'))

aggr_model.compile(optimizer=kr.optimizers.Adam(learning_rate=EMBED_LEARNING_RATE), loss='binary_crossentropy',
                 metrics=[kr.metrics.BinaryAccuracy(), kr.metrics.Precision(), kr.metrics.Recall(), kr.metrics.F1Score()])
aggr_model.fit(X_train_aggr, y_train, epochs=EMBED_NUM_EPOCHS, validation_data=(X_val_aggr, y_val),
              callbacks=[kr.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=2, start_from_epoch=5)])




Set length is 18256
Set length is 2931
Set length is 5129
[ 1.50160333e-01  5.44801234e-02  2.41527209e-02  1.05055335e-01
 -7.45249366e-02  1.31290580e-01  5.96303948e-02 -3.85678927e-02
  3.89007756e-02 -6.62222672e-02 -1.00826310e-02 -2.22939725e-01
 -3.53781233e+00 -1.51060405e-02 -1.17666260e-02 -1.20102991e-01
 -5.11143730e-02 -9.43388302e-02 -1.46557187e-01  1.97243609e-02
  1.65357753e-01  1.03652613e-02 -7.15417665e-02 -7.07047536e-02
 -6.30843680e-02  1.89529388e-01 -4.52385026e-02 -1.93568563e-02
 -1.64860791e-02 -3.38175528e-02  7.23086583e-02  6.90387256e-02
 -1.41813176e-01  3.60366876e-02  1.66785376e-02 -3.86980695e-03
  2.82496364e-02  3.47522123e-02 -2.11887658e-02 -1.10657968e-01
 -2.67682449e-01 -8.76868284e-02  1.34778466e-01  4.94314591e-02
  1.97006930e-01 -6.69918830e-02  1.19804438e-01 -5.98159466e-02
 -1.01369395e-01  4.08873848e-02 -6.43136737e-02  7.41605643e-02
 -3.17177741e-02  7.76113833e-03 -4.86032200e-02  1.02473722e-01
 -1.93872005e-01 -4.59208693e-02

This is my current version of my LSTM model. It isn't as good as I'd like it to be, with an hour-long training time and accuracy that seems to be still worse than my initial model. After a lot of difficulty with both RAM and getting Keras to accept variable-length input (the entire point of an RNN), I settled on an approach where Keras would train an embedding on my model while training the model.

Unfortunately, it doesn't seem to be making a very good embedding or model. It is better than my second model, but still fails to my first.

Interestingly, it looks like Keras has an optimization whereby it can use a graphics card to tremendously speed performance.

I've noticed that all my models seem to have about the same precision as recall.

In [21]:
# Model version 2, using their embedding
max_len = 0

def sentence_to_ids(sentence_list):
    global max_len
    tokens = [list(gs.utils.tokenize(sentence, lowercase=True, deacc=True)) for sentence in sentence_list]
    if max_len == 0:
        max_len = max((len(review) for review in tokens))
    return kr.preprocessing.sequence.pad_sequences([[vectorizer.vocabulary_[word] if word in vectorizer.vocabulary_ else 0. for word in sentence]
            for sentence in tokens], maxlen=max_len).astype('int32')
    # I'm dropping words outside the vocabulary, hopefully that doesn't cause problems

# Now I convert everything
X_train_lstm = sentence_to_ids(X_train)
print("Done with train")
X_val_lstm = sentence_to_ids(X_val)
X_test_lstm = sentence_to_ids(X_test)


# now I make my model
# LSTM_FEATURE_SHAPE = X_train_lstm[0].shape
LSTM_LEARNING_RATE = 0.0001
LSTM_NUM_EPOCHS = 50

lstm_model = kr.models.Sequential()
# lstm_model.add(kr.Input(shape=(None, max_len)))
lstm_model.add(kr.layers.Masking(mask_value=0))
lstm_model.add(kr.layers.Embedding(len(vectorizer.vocabulary_), 50))
lstm_model.add(kr.layers.LSTM(10))
lstm_model.add(kr.layers.Dense(1, activation='sigmoid'))

# y_train_lstm = y_train.reshape((1, len(y_train)))
# y_val_lstm = y_val.reshape((1, len(y_val)))

lstm_model.compile(optimizer=kr.optimizers.Adam(learning_rate=LSTM_LEARNING_RATE), loss='binary_crossentropy',
                 metrics=[kr.metrics.BinaryAccuracy(), kr.metrics.Precision(), kr.metrics.Recall(), kr.metrics.F1Score()])
lstm_model.fit(X_train_lstm, y_train, epochs=LSTM_NUM_EPOCHS, validation_data=(X_val_lstm, y_val),
              callbacks=[kr.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=0, start_from_epoch=1)])

Done with train
Epoch 1/50
625/625 [==============================] - 94s 147ms/step - loss: 0.6719 - binary_accuracy: 0.6126 - precision_8: 0.5774 - recall_8: 0.8521 - f1_score: 0.6686 - val_loss: 0.5689 - val_binary_accuracy: 0.7798 - val_precision_8: 0.7466 - val_recall_8: 0.8351 - val_f1_score: 0.6588
Epoch 2/50
625/625 [==============================] - 68s 109ms/step - loss: 0.5083 - binary_accuracy: 0.8084 - precision_8: 0.7885 - recall_8: 0.8453 - f1_score: 0.6686 - val_loss: 0.4590 - val_binary_accuracy: 0.8438 - val_precision_8: 0.8373 - val_recall_8: 0.8465 - val_f1_score: 0.6588
Epoch 3/50
625/625 [==============================] - 60s 97ms/step - loss: 0.3901 - binary_accuracy: 0.8763 - precision_8: 0.8864 - recall_8: 0.8644 - f1_score: 0.6686 - val_loss: 0.3938 - val_binary_accuracy: 0.8666 - val_precision_8: 0.8655 - val_recall_8: 0.8624 - val_f1_score: 0.6588
Epoch 4/50
625/625 [==============================] - 54s 86ms/step - loss: 0.3217 - binary_accuracy: 0.9025 - p

Below are the test runs of each model. Because there f1 score doesn't seem to work, I also calculated f1 manually. Interestingly, though my first model has the best precision, my last one has the best recall.

In [10]:
results = bow_model.evaluate(X_test_bow, y_test)
print(f'Calculated f1 score: {2* results[2] * results[3] / (results[2] + results[3])}')

782/782 [==============================] - 6s 7ms/step - loss: 0.3483 - binary_accuracy: 0.8734 - precision_1: 0.8891 - recall_1: 0.8533 - f1_score: 0.6685
Calculated f1 score: 0.8708360533720181


In [27]:
results = aggr_model.evaluate(X_test_aggr, y_test)
print(f'Calculated f1 score: {2* results[2] * results[3] / (results[2] + results[3])}')

782/782 [==============================] - 2s 3ms/step - loss: 0.4028 - binary_accuracy: 0.8153 - precision_12: 0.8047 - recall_12: 0.8327 - f1_score: 0.6667
Calculated f1 score: 0.8184784577662271


In [22]:
results = lstm_model.evaluate(X_test_lstm, y_test)
print(f'Calculated f1 score: {2* results[2] * results[3] / (results[2] + results[3])}')

782/782 [==============================] - 24s 30ms/step - loss: 0.3745 - binary_accuracy: 0.8576 - precision_8: 0.8280 - recall_8: 0.9029 - f1_score: 0.6667
Calculated f1 score: 0.8638016264840869
